In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Train dataframe
df_train = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "message"])
df_train.dropna()
df_train.head()

In [ ]:
# Test dataframe
df_test = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "message"])
df_test.dropna()
df_test.head()

In [ ]:
df_train["type"] = pd.factorize(df_train["type"])[0]
df_test["type"] = pd.factorize(df_test["type"])[0]
df_train.head()

In [ ]:
train_labels = df_train["type"].values
ds_train = tf.data.Dataset.from_tensor_slices(
              (df_train["message"].values, 
               train_labels))

test_labels =  df_test["type"].values
ds_test = tf.data.Dataset.from_tensor_slices(
              (df_test["message"].values, 
               test_labels))

In [ ]:
ds_train = ds_train.shuffle(100).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vector = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000)

vector.adapt(ds_train.map(lambda text, label: text))

In [ ]:
vocab = np.array(vector.get_vocabulary())
vocab[:15]

In [ ]:
model = tf.keras.Sequential([
    vector,
    tf.keras.layers.Embedding(len(vector.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'],
)

model.summary()

In [ ]:
history = model.fit(
            ds_train,
            epochs=3,
            validation_data=ds_test,
            validation_steps=10)

In [ ]:
test_loss, test_acc = model.evaluate(ds_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
def plot_graphs(i, metric):
    plt.plot(i.history[metric])
    plt.plot(i.history['val_'+metric])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
i = history.history
print(i['loss'])
print(i['val_loss'])
print(i['accuracy'])
print(i['val_accuracy'])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    ps = model.predict([pred_text])
    print(ps)
    p = ps[0][0]
    return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
